## Newton Method

### Quadratic Energy Minimization
We are minimizing the following energy functional, using a Netwon method based on the TF Hessian library.
$$J(x,y) = x^2y^2+xy$$

which is the unique stationary point of $\nabla J$ given the fact that $J(x,y)$ is convex.

In [1]:
#We import all the library we are gona need
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from numsa.TFHessian import *

In [2]:
itmax = 10; # Number of epoch.
tol = 1e-8
step_size = 1; #Learning rate
def Loss(x):
    return (x[0]**2)*(x[1]**2)+x[0]*x[1];
#Defining the Hessian class for the above loss function in x0
x = tf.Variable(0.1*np.ones((2,1),dtype=np.float32))
H =  Hessian(Loss,x)
grad = H.grad().numpy();
print("Lost funciton at this iteration {}, gradient norm {} and is achived at point {}"
      .format(Loss(x),np.linalg.norm(grad),x));
print("Computed the  first gradient ...")
q = H.pCG(grad,1,1,tol=tol,itmax=100);
print("Computed search search diratcion ...")
print("Entering the Netwton optimization loop")
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%50 == 0:
        print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        print("Lost funciton at this iteration {}, gradient norm {} and is achived at point {}"
      .format(Loss(x),np.linalg.norm(grad),x));
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    q = H.pCG(grad,1,1,tol=tol,itmax=100);

Lost funciton at this iteration [0.0101], gradient norm 0.1442497819662094 and is achived at point <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[0.1],
       [0.1]], dtype=float32)>
Computed the  first gradient ...


 20%|██        | 2/10 [00:00<00:00, 13.46it/s]

Computed search search diratcion ...
Entering the Netwton optimization loop
Lost funciton at this iteration [1.4239612e-05]  and gradient norm 0.1442497819662094


 90%|█████████ | 9/10 [00:00<00:00, 15.13it/s]

Lost funciton at this iteration [8.984099e-16], gradient norm 9.324332417293135e-09 and is achived at point <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[1.8767974e-08],
       [4.7869307e-08]], dtype=float32)>


### Regression
Our objective is to minimize the function:
\begin{equation}
    f(\vec{x}) = \frac{1}{m} \sum_{i=1}^m \log\Bigg(1+\exp\Big(-b_j \vec{a_j}^T\vec{x}\Big)\Bigg)\qquad for \; x \in \mathbb{R}^d
\end{equation}

where $d$ is the feature number and $\vec{a}_j$ are the data while $b_j$ are the labels.
Now we would like to this applying the newton method to find a point that minimize such a function. This is possible because since $f$ is convex, all stationary points are minimizers and we search for the "roots" of the equation $\nabla f=0$.
The newton method we implement is of the form,
\begin{equation}
    \vec{x}_{n+1} = \vec{x}_n -\gamma Hf(\vec{x}_n)^{-1}\nabla f(\vec{x}_n)
\end{equation}

where $\gamma$ is the step size.
We solve the system $Hf(\vec{x}_n)q=\nabla f(\vec{x}_n)$ using the CG method where as a preconditioned we have taken a the inverse of $Hf(\vec{x}_n)$ computed using the random SVD presented in [1].

In [3]:
#We import all the library we are gona need
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from numsa.TFHessian import *
import dsdl

In [4]:
ds = dsdl.load("a1a")

X, Y = ds.get_train()
print(X.shape, Y.shape)

(1605, 119) (1605,)


In [ ]:
#Setting the parameter of this run, we will use optimization nomeclature not ML one.
itmax = 100; # Number of epoch.
tol = 1e-4
step_size = 0.2; #Learning rate
Err = [];
"""
#Old Lost Function, intesead using Stefano's.
#Defining the Loss Function
def Loss(x):
    S = tf.Variable(0.0);
    for j in range(X.shape[0]):
        a = tf.constant((X[j,:].todense().reshape(119,1)),dtype=np.float32);
        b = tf.constant(Y[j],dtype=np.float32)
        a = tf.reshape(a,(119,1));
        x = tf.reshape(x,(119,1));
        dot = tf.matmul(tf.transpose(a),x);
        S = S+tf.math.log(1+tf.math.exp(-b*dot))
    S = (1/X.shape[0])*S;
    return S;
"""
tfX = tf.sparse.from_dense(np.array(X.todense(), dtype=np.float32))
tfY = tf.convert_to_tensor(np.array(Y, dtype=np.float32).reshape(X.shape[0], 1))


#Defining the Loss Function
def Loss(x):
    x = tf.reshape(x, (119, 1))
    Z = tf.sparse.sparse_dense_matmul(tfX, x, adjoint_a=False)
    Z = tf.math.multiply(tfY, Z)
    S = tf.reduce_sum(tf.math.log(1 + tf.math.exp(-Z)) / tfX.shape[0])
    return S

#Defining the Hessian class for the above loss function in x0
x = tf.Variable(0.1*np.ones((119,1),dtype=np.float32))
H =  Hessian(Loss,x)
grad = H.grad().numpy();
print("Computed the  first gradient ...")
q = grad #H.pCG(grad,10,2,tol=1e-3,itmax=10);
print("Computed search search diratcion ...")
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%50 == 0:
        print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    q = grad #H.pCG(grad,10,2,tol=1e-3,itmax=10);
itmax = 100; # Number of epoch.
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    Err = Err + [np.linalg.norm(grad)];
    if it%10 == 0:
        print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    q = H.pCG(grad,65,10,tol=1e-4,itmax=100);

 24%|██▍       | 24/100 [00:00<00:00, 233.59it/s]

Computed the  first gradient ...
Computed search search diratcion ...
Lost funciton at this iteration 0.9243690371513367  and gradient norm 1.3872039318084717


 95%|█████████▌| 95/100 [00:00<00:00, 226.93it/s]

Lost funciton at this iteration 0.3988267183303833  and gradient norm 0.07041343301534653


  0%|          | 0/100 [00:00<?, ?it/s]

Lost funciton at this iteration 0.37019962072372437  and gradient norm 0.041932351887226105


  2%|▏         | 2/100 [00:07<06:03,  3.70s/it]

In [ ]:
print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));

### Quasi-Newton Method

In [ ]:
#We import all the library we are gona need
import tensorflow as tf
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
from numsa.TFHessian import *
import dsdl

In [ ]:
ds = dsdl.load("a1a")

X, Y = ds.get_train()
print(X.shape, Y.shape)

In [ ]:
#Setting the parameter of this run, we will use optimization nomeclature not ML one.
itmax = 100; # Number of epoch.
tol = 1e-4
step_size = 0.2; #Learning rate
Err = [];
Hs = [];
Rsigmas50 = [];
Rsigmas80 = [];
"""
#Old Lost Function, intesead using Stefano's.
#Defining the Loss Function
def Loss(x):
    S = tf.Variable(0.0);
    for j in range(X.shape[0]):
        a = tf.constant((X[j,:].todense().reshape(119,1)),dtype=np.float32);
        b = tf.constant(Y[j],dtype=np.float32)
        a = tf.reshape(a,(119,1));
        x = tf.reshape(x,(119,1));
        dot = tf.matmul(tf.transpose(a),x);
        S = S+tf.math.log(1+tf.math.exp(-b*dot))
    S = (1/X.shape[0])*S;
    return S;
"""
tfX = tf.sparse.from_dense(np.array(X.todense(), dtype=np.float32))
tfY = tf.convert_to_tensor(np.array(Y, dtype=np.float32).reshape(X.shape[0], 1))


#Defining the Loss Function
def Loss(x):
    x = tf.reshape(x, (119, 1))
    Z = tf.sparse.sparse_dense_matmul(tfX, x, adjoint_a=False)
    Z = tf.math.multiply(tfY, Z)
    S = tf.reduce_sum(tf.math.log(1 + tf.math.exp(-Z)) / tfX.shape[0])
    return S
#Defining the Hessian class for the above loss function in x0
x = tf.Variable(0.1*np.ones((119,1),dtype=np.float32))
H =  Hessian(Loss,x)
grad = H.grad().numpy();
print("Computed the  first gradient ...")
q = grad #H.pCG(grad,10,2,tol=1e-3,itmax=10);
print("Computed search search diratcion ...")
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%50 == 0:
        print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    q = grad #H.pCG(grad,10,2,tol=1e-3,itmax=10);
itmax = 100; # Number of epoch.
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    Err = Err + [np.linalg.norm(grad)];
    if it%5 == 0:
        print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));
        Hs = Hs + [H.mat()];
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    U, s, Vt = H.RandMatSVD(50,10)
    if it%5 == 0:
        Rsigmas50 = Rsigmas50 + [s];
    U, s, Vt = H.RandMatSVD(80,10)
    if it%5 == 0:
        Rsigmas80 = Rsigmas80 + [s];
    q = (Vt.transpose()@np.linalg.inv(np.diag(s))@U.transpose())@grad;

In [ ]:
np.save("Hs",Hs)
print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x),np.linalg.norm(grad)));

We would like to make a case for using the random SVD alghorithm in second order optimization problem. Let us consider the Hessian produced during in the in pevious example and show that given their quick decay the randomised SVD is a good approximation.

In [ ]:
Hs = np.load("Hs.npy")
plt.figure()
for H in Hs:
    _, s,_ = np.linalg.svd(H)
    plt.semilogy(s)
plt.legend([r"$H_{"+str(5*i)+"}$" for i in range(len(Hs))])
plt.title("Singular Value Hessian Regressian For Adult Problem")

In [ ]:
plt.figure()
_, s,_ = np.linalg.svd(Hs[4])
plt.semilogy(s)
plt.semilogy(Rsigmas50[4],"--")
plt.semilogy(Rsigmas80[4],"--")
plt.legend([r"$H_{4}$",r"$\overset{\sim}{H}_4^{50}$",r"$\overset{\sim}{H}_4^{80}$"])
plt.title("Singular Value Hessian Regressian For Adult Problem")

In [ ]:
from ipyparallel import Client
c = Client()
c.ids

In [ ]:
%%px 
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numsa.TFHessian import *
from mpi4py import MPI
import dsdl

In [ ]:
%%px
itmax = 100; # Number of epoch.
tol = 1e-4
step_size = 0.8; #Learning rate

In [ ]:
%%px
comm = MPI.COMM_WORLD

ds = dsdl.load("a1a")

X, Y = ds.get_train()
indx = np.array_split(range(X.shape[0]),int(comm.Get_size()));
tfX = []
tfY = []
for k in range(len(indx)):
    tfX = tfX + [tf.sparse.from_dense(np.array(X[indx[comm.Get_rank()]].todense(), dtype=np.float32))]
    tfY = tfY + [tf.convert_to_tensor(np.array(Y[indx[comm.Get_rank()]], dtype=np.float32).reshape(X[indx[comm.Get_rank()]].shape[0], 1))]
#Defining the Loss Function
def Loss(x,comm):
    x = tf.reshape(x, (119, 1))
    Z = tf.sparse.sparse_dense_matmul(tfX[comm.Get_rank()], x, adjoint_a=False)
    Z = tf.math.multiply(tfY[comm.Get_rank()], Z)
    S = tf.reduce_sum(tf.math.log(1 + tf.math.exp(-Z)) / tfX[comm.Get_rank()].shape[0])
    return S

In [ ]:
%%px
x = tf.Variable(0.1*np.ones((119,1),dtype=np.float32))
H =  Hessian(Loss,x)
grad = H.grad().numpy();
q = grad
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%50 == 0:
        print("[Iteration. {}] Lost funciton at this iteration {}  and gradient norm {}".format(it,Loss(x,H.comm),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    q = grad #H.pCG(grad,10,2,tol=1e-3,itmax=10);
    
itmax = 200
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%5 == 0:
        print("[Iteration. {}] Lost funciton at this iteration {}  and gradient norm {}".format(it,Loss(x,H.comm),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    U, s, Vt = H.RandMatSVD(65,10)
    q = (Vt.transpose()@np.linalg.inv(np.diag(s))@U.transpose())@grad;
print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x,H.comm),np.linalg.norm(grad)));

### Federated Newton Learn
In this section we present the algorithm named Federated Newton Learning (FEDNL) introduced in [2].

In [1]:
from ipyparallel import Client
c = Client()
c.ids

[0, 1]

In [2]:
%%px
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numsa.TFHessian import *
import dsdl

comm = MPI.COMM_WORLD

ds = dsdl.load("a1a")

X, Y = ds.get_train()
indx = np.array_split(range(X.shape[0]),int(comm.Get_size()));
tfX = []
tfY = []
for k in range(len(indx)):
    tfX = tfX + [tf.sparse.from_dense(np.array(X[indx[comm.Get_rank()]].todense(), dtype=np.float32))]
    tfY = tfY + [tf.convert_to_tensor(np.array(Y[indx[comm.Get_rank()]], dtype=np.float32).reshape(X[indx[comm.Get_rank()]].shape[0], 1))]
#Defining the Loss Function
def Loss(x,comm):
    x = tf.reshape(x, (119, 1))
    Z = tf.sparse.sparse_dense_matmul(tfX[comm.Get_rank()], x, adjoint_a=False)
    Z = tf.math.multiply(tfY[comm.Get_rank()], Z)
    S = tf.reduce_sum(tf.math.log(1 + tf.math.exp(-Z)) / tfX[comm.Get_rank()].shape[0])
    return S

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

In [ ]:
%%px
#Setting the parameter of this run, we will use optimization nomeclature not ML one.
itmax = 100; # Number of epoch.
tol = 1e-4
step_size = 0.2; #Learning rate
x = tf.Variable(0.1*np.ones((119,1),dtype=np.float32))
H =  Hessian(Loss,x)
grad = H.grad().numpy();
q = grad
for it in tqdm(range(itmax)):
    x = x - tf.constant(step_size,dtype=np.float32)*tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%50 == 0:
        print("[Iteration. {}] Lost funciton at this iteration {}  and gradient norm {}".format(it,Loss(x,H.comm),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    H =  Hessian(Loss,x)
    grad = H.grad().numpy();
    q = grad #H.pCG(grad,10,2,tol=1e-3,itmax=10);
    
itmax = 200
for it in tqdm(range(itmax)):
    x = x - tf.Variable(q,dtype=np.float32);
    x =  tf.Variable(x)
    if it%5 == 0:
        print("[Iteration. {}] Lost funciton at this iteration {}  and gradient norm {}".format(it,Loss(x,H.comm),np.linalg.norm(grad)));
    if np.linalg.norm(grad)<tol:
        break
    U, s, Vt = H.shift(x,{"comp":SingularCompression,"rk":1})
    grad = H.grad().numpy();
    H.Hs = H.comm.gather(H.memH, root=0);
    if H.comm.Get_rank() == 0:
        Hm = (1/len(H.Hs))*np.sum(H.Hs,0);
        H.memH = Hm;
        H.mem = (H.memH+step_size*(Vt.transpose()@np.linalg.inv(np.diag(s))@U.transpose()));
        q = H.memH@grad;
        H.comm.bcast(q,root=0)
    else:
        H.Hs = None
        H.mem = (H.memH+step_size*(Vt.transpose()@np.linalg.inv(np.diag(s))@U.transpose()));
print("Lost funciton at this iteration {}  and gradient norm {}".format(Loss(x,H.comm),np.linalg.norm(grad)))

 12%|█▎        | 25/200 [01:39<11:31,  3.95s/it]

 13%|█▎        | 26/200 [01:41<11:29,  3.96s/it]

[stdout:0] [Iteration. 0] Lost funciton at this iteration 0.9227030873298645  and gradient norm 1.3872549533843994
[Iteration. 50] Lost funciton at this iteration 0.4135685861110687  and gradient norm 0.06812936067581177
[Iteration. 0] Lost funciton at this iteration 0.3849295675754547  and gradient norm 0.04145266115665436
[Iteration. 5] Lost funciton at this iteration 0.3779388666152954  and gradient norm 0.03542470932006836
[Iteration. 10] Lost funciton at this iteration 0.37261834740638733  and gradient norm 0.031228117644786835
[Iteration. 15] Lost funciton at this iteration 0.3683735728263855  and gradient norm 0.028081871569156647
[Iteration. 20] Lost funciton at this iteration 0.3648774027824402  and gradient norm 0.025608040392398834
[Iteration. 25] Lost funciton at this iteration 0.36192959547042847  and gradient norm 0.0236001405864954


[stdout:1] [Iteration. 0] Lost funciton at this iteration 0.9254758358001709  and gradient norm 1.3883233070373535
[Iteration. 50] Lost funciton at this iteration 0.3762178122997284  and gradient norm 0.07581852376461029
[Iteration. 0] Lost funciton at this iteration 0.3408527672290802  and gradient norm 0.04587741196155548
[Iteration. 5] Lost funciton at this iteration 0.33158451318740845  and gradient norm 0.03843647614121437
[Iteration. 10] Lost funciton at this iteration 0.3239428400993347  and gradient norm 0.03257639706134796
[Iteration. 15] Lost funciton at this iteration 0.31773948669433594  and gradient norm 0.028391094878315926
[Iteration. 20] Lost funciton at this iteration 0.312810480594635  and gradient norm 0.025993244722485542
[Iteration. 25] Lost funciton at this iteration 0.30901283025741577  and gradient norm 0.025357605889439583


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]